## Classification pipeline

In [ ]:
# imports
import numpy as np
import pandas as pd
import pickle
import mne
from mne_bids import BIDSPath, read_raw_bids
from typing import List
import xarray as xr
from pathlib import Path
from scipy.signal import resample, butter, sosfiltfilt, iirnotch, tf2sos

## Preprocessing OTKA

In [22]:
def process_input(path: str,
                  subject: str,
                  task: str,
                  channels: List[str],
                  resampling_frq=200,
                  notchfilter_frq=50,
                  filter_bounds=[0.3, 75],
                  verbose=False):

    # open data
    print(f'>>>>>>>>preprocessing {subject}-{task}')
    bids_path = BIDSPath(subject=subject,
                         session='01',
                         task=task,
                         root=path
                         )
    raw = read_raw_bids(bids_path, extra_params={'preload': True}, verbose=verbose)

    # pick eeg channels
    print('picking eeg channels...')
    # ['T3', 'T4', 'T5', 'T6'] channels that are only in the 10-20 system
    # replaced with their equivalent name in the 10-10 system [T7, T8, P7, P8]
    raw.pick(channels, verbose=verbose)

    # interpolate bad channels if there is any
    print('interpolating bad channels...')
    raw.interpolate_bads(verbose=verbose)

    # resampling
    if resample is not None:
        raw.resample(resampling_frq, verbose=verbose)

    raw.filter(l_freq=filter_bounds[0], h_freq=filter_bounds[1], verbose=verbose)
    raw.notch_filter((notchfilter_frq), verbose=verbose)
    eeg_array = raw.get_data().T * 10**6  # volts to microvolts
    points, chs = eeg_array.shape
    a = points % (30 * 200)
    eeg_array = eeg_array[60 * 200:-(a+60 * 200), :]
    eeg_array = eeg_array.reshape(-1, 30, 200, chs)
    eeg_array = eeg_array.transpose(0, 3, 1, 2)

    return eeg_array

In [ ]:
eeg_path = '/Volumes/Extreme_SSD/PhD/OTKA_study1/EEG_data/BIDS/'
behvioral_path = '../EEGModalNet/data/OTKA/PLB_HYP_data_MASTER.csv'

behavioral = pd.read_csv(behvioral_path)
gender = behavioral['gender']
hypnotizability = behavioral['hypnotizability_total']
bid_id = behavioral['bids_id']

In [68]:
subjects = [f'{i:02}' for i in range(1, 3)]
tasks = ['experience1', 'experience2', 'experience3', 'experience4']
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz']

def load_trial(path, subj, task):
    eeg_data = process_input(
        path=path,
        subject=subj,
        task=task,
        channels=channels,
        verbose=0
    )
    return eeg_data

def load_subject_trials(path, subj, tasks):
    return np.vstack(np.array([load_trial(path, subj, task) for task in tasks]))

all_eeg_data = [load_subject_trials(eeg_path, subj, tasks) for subj in subjects]

>>>>>>>>preprocessing 01-experience1
picking eeg channels...
interpolating bad channels...


/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_67174/868689552.py:27: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(verbose=verbose)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


>>>>>>>>preprocessing 01-experience2
picking eeg channels...
interpolating bad channels...


/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_67174/868689552.py:27: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(verbose=verbose)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


>>>>>>>>preprocessing 01-experience3
picking eeg channels...
interpolating bad channels...


...

/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_67174/868689552.py:27: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(verbose=verbose)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


>>>>>>>>preprocessing 02-experience3
picking eeg channels...
interpolating bad channels...


/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_67174/868689552.py:27: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(verbose=verbose)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


>>>>>>>>preprocessing 02-experience4
picking eeg channels...
interpolating bad channels...


/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_67174/868689552.py:27: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads(verbose=verbose)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [70]:
segments_to_be_excluded = {}

for sub in range(len(all_eeg_data)):
    sample_key = []
    for i, sample, in enumerate(all_eeg_data[sub]):
        if np.max(np.abs(sample)) > 100:
            sample_key.append(i)
    segments_to_be_excluded[f'sub_{sub}'] = sample_key

# exclude bad epochs
filtered_x = []

for sub in range(len(all_eeg_data)):
    excluded_segments = segments_to_be_excluded[f'sub_{sub}']
    filtered_sub = np.delete(all_eeg_data[sub], excluded_segments, axis=0)  # Remove excluded segments
    filtered_x.append(filtered_sub)

# Convert the filtered list back to a numpy array
x = np.array(filtered_x, dtype=object)

In [74]:
x[1].shape

(16, 19, 30, 200)

In [ ]:
behvioral_path = '../EEGModalNet/data/OTKA/PLB_HYP_data_MASTER.csv'

behavioral = pd.read_csv(behvioral_path).dropna(subset='bids_id')
behavioral['bids_id'] = behavioral['bids_id'].apply(lambda x: f'{int(x):02}')
bid_id = behavioral['bids_id'].values
behavioral.set_index('bids_id', inplace=True)
gender = behavioral['gender']
hypnotizability = behavioral['hypnotizability_total']
# bid_id = behavioral['bids_id']

In [82]:
all_segments = []
subject_ids = []
epoch_ids = []
gender_of_epoch = []
hypnotizablity_of_epoch = []

for subj_idx, subj_array in zip(subjects, x):
    for epoch_idx, segment in enumerate(subj_array):
        all_segments.append(segment)
        subject_ids.append(subj_idx)
        gender_of_epoch.append(gender[subj_idx])
        hypnotizablity_of_epoch.append(int(hypnotizability[subj_idx]))
        epoch_ids.append(epoch_idx)

# Step 2: Convert list to array
all_segments = np.stack(all_segments)  # shape (total_epochs, 30, 128)

subject_epoch = [f'{sub_id}_epoch-{epo_id}' for sub_id, epo_id in zip(subject_ids, epoch_ids)]

data = xr.DataArray(
    all_segments,
    dims=("subject_epoch", "channel", "segment", "time"),
    coords={
        "subject_epoch": subject_epoch,
        "channel": channels,
        "segment": np.arange(all_segments.shape[2]),
        "time": np.arange(all_segments.shape[3]),
    },
    attrs={'gender': gender_of_epoch,
           'hypnotizablity': hypnotizablity_of_epoch},
    name="eeg"
)

In [83]:
data

<xarray.DataArray 'eeg' (subject_epoch: 44, channel: 19, segment: 30, time: 200)> Size: 40MB
array([[[[-4.52276396e+00, -8.06634156e+00, -1.30717939e+01, ...,
          -1.51840130e+01, -1.17841298e+01, -9.03270520e+00],
         [-7.20169361e+00, -9.27297983e+00, -8.11781416e+00, ...,
          -1.57985228e+01, -1.35829837e+01, -2.12534965e+01],
         [-1.97145199e+01, -1.13542852e+01, -6.29209078e+00, ...,
           2.30171311e+01,  1.61016616e+01,  2.18512923e+01],
         ...,
         [ 1.03890558e+01,  7.89506886e+00,  3.93571694e+00, ...,
          -6.34172331e+00, -5.77984031e+00, -5.07813437e+00],
         [-2.59696000e+00, -3.13327075e+00, -3.43676186e+00, ...,
           6.40709286e-01, -1.48836715e+00, -7.72200519e+00],
         [-1.50941376e+01, -1.81145947e+01, -1.18473243e+01, ...,
           2.28869487e+00, -5.38436237e+00, -5.72759498e-01]],

        [[-1.03318562e+01, -7.94414134e+00, -7.77746761e+00, ...,
          -1.27171529e+00, -1.28196902e+01, -5.63477301e+00],
         [ 1.41811230e+00, -1.06514924e+01, -6.87436444e+00, ...,
          -1.36453514e+01, -4.52986018e+00, -1.14050137e+01],
         [-1.31950536e+01, -2.26868635e+00,  4.88262860e+00, ...,
           1.95987811e+01,  1.88167120e+01,  1.15171481e+01],
...
         [-1.26569726e+00, -7.32227957e-01,  5.10126351e-02, ...,
          -4.00081961e+00, -3.48904736e+00, -3.29085766e+00],
         [-3.82932420e+00, -4.69062390e+00, -4.39085107e+00, ...,
           1.18270302e+00,  1.38430024e+00,  8.06958453e-01],
         [ 3.94022315e-01,  1.99072676e+00,  3.42928950e+00, ...,
          -2.66878087e+00, -2.67078199e+00, -1.26193939e+00]],

        [[ 2.57538342e+00, -1.43526447e+00, -6.40719676e+00, ...,
          -1.56662356e+01, -2.63824180e+01, -2.86003623e+01],
         [-2.38147519e+01, -1.82913496e+01, -1.62987069e+01, ...,
           2.64273287e+01,  2.23899325e+01,  1.67920089e+01],
         [ 6.06153513e+00, -4.53623028e+00, -1.09854203e+01, ...,
           4.14419733e-01,  1.90521891e+00,  7.72885607e-01],
         ...,
         [ 1.60390133e+00,  5.83205305e-01,  3.12150383e+00, ...,
          -1.53788584e+01, -1.39321491e+01, -1.22395073e+01],
         [-9.97607907e+00, -7.15507413e+00, -6.41177081e+00, ...,
          -1.04732895e+01, -9.66591671e+00, -7.77938009e+00],
         [-4.43866953e+00,  3.45663459e+00,  7.94727501e+00, ...,
          -1.58279752e+00,  5.25639708e-01,  4.93772843e+00]]]])
Coordinates:
  * subject_epoch  (subject_epoch) <U11 2kB '01_epoch-0' ... '02_epoch-15'
  * channel        (channel) <U3 228B 'Fp1' 'Fp2' 'F3' 'F4' ... 'Fz' 'Cz' 'Pz'
  * segment        (segment) int64 240B 0 1 2 3 4 5 6 7 ... 23 24 25 26 27 28 29
  * time           (time) int64 2kB 0 1 2 3 4 5 6 ... 194 195 196 197 198 199
Attributes:
    gender:          ['Female', 'Female', 'Female', 'Female', 'Female', 'Fema...
    hypnotizablity:  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5...

In [ ]:
# feature extraction
import torch 
from CBraMod.models.cbramod import CBraMod
from einops.layers.torch import Rearrange

backbone = CBraMod(
    in_dim=200, out_dim=200, d_model=200,
    dim_feedforward=800, seq_len=30,
    n_layer=12, nhead=8
)

backbone.load_state_dict(
    torch.load(params.foundation_dir,
                map_location=torch.device(params.device)))
backbone.proj_out = nn.Identity()

# bz, ch_num, seq_len, patch_size = x.shape

features = backbone(x)  # TODO x
features = Rearrange('b c s p -> b (c s p)')(features).contiguous()
features = features.flatten(1)
print("features.shape=", features.shape)

# # store features
# torch.save({'features': features.detach().numpy(),
#             'gender': ds.y,
#             'subject_ids': ds.subject_ids}, params.features_file_path)
# print("features saved to", params.features_file_path)

## preprocessing Lemon

In [ ]:
# open lemon data

# channels: ['T3', 'T4', 'T5', 'T6'] channels that are only in the 10-20 system replaced with their equivalent name in the 10-10 system [T7, T8, P7, P8].
channels = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3',
            'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']  # FIXME: odering might be different in the data they inputed to their model

n_subjects = 202
lemon = xr.open_dataset('data/LEMON/eeg_eo_ec.nc5')

x = lemon['eye_closed'].sel(subject=lemon.subject[:n_subjects], channel=channels).to_numpy() * 10**6  # Volts to microvolts

# FIXME resampling to 200Hz
# n_samples = int((x.shape[-1] / sampling_rate) * downsample_frq)
# x = resample(x, num=n_samples, axis=-1)

# bandpass filter
sos = butter(4, 0.3, btype='highpass', fs=128, output='sos')
x = sosfiltfilt(sos, x, axis=-1)

# notchfilter
fs = 128
f0 = 50
Q = 30
w0 = f0 / (fs / 2)  # Normalized Frequency
b, a = iirnotch(w0, Q)
sos = tf2sos(b, a)
x = sosfiltfilt(sos, x, axis=-1)

subs, chs, points = x.shape

# epoching
x = x.reshape(subs, -1, chs, 30, 128)  # subjects, segments, channels, 30 seconds, sampling rate

In [ ]:
segments_to_be_excluded = {}

for sub in range(x.shape[0]):
    sample_key = []
    for i, sample, in enumerate(x[sub]):
        if np.max(np.abs(sample)) > 100:
            sample_key.append(i)
    segments_to_be_excluded[f'sub_{sub}'] = sample_key

# exclude bad epochs
filtered_x = []

for sub in range(x.shape[0]):
    excluded_segments = segments_to_be_excluded[f'sub_{sub}']
    filtered_sub = np.delete(x[sub], excluded_segments, axis=0)  # Remove excluded segments
    filtered_x.append(filtered_sub)

# Convert the filtered list back to a numpy array
x = np.array(filtered_x, dtype=object)

In [ ]:
demog = pd.read_csv('data/LEMON/Demographics.csv')
gender = demog['Gender_ 1=female_2=male'].values
age = demog['Age']

In [ ]:
all_segments = []
subject_ids = []
epoch_ids = []
gender_of_epoch = []
age_of_epoch = []

for subj_idx, subj_array in enumerate(x):
    for epoch_idx, segment in enumerate(subj_array):
        all_segments.append(segment)
        subject_ids.append(lemon.subject.values.tolist()[subj_idx])
        gender_of_epoch.append(int(gender[subj_idx]))
        age_of_epoch.append(age[subj_idx])
        epoch_ids.append(epoch_idx)

# Step 2: Convert list to array
all_segments = np.stack(all_segments)  # shape (total_epochs, 30, 128)

subject_epoch = [f'{sub_id}_epoch-{epo_id}' for sub_id, epo_id in zip(subject_ids, epoch_ids)]

data = xr.DataArray(
    all_segments,
    dims=("subject_epoch", "channel", "segment", "time"),
    coords={
        "subject_epoch": subject_epoch,
        "channel": channels,
        "segment": np.arange(all_segments.shape[2]),
        "time": np.arange(all_segments.shape[3]),
    },
    attrs={'gender': gender_of_epoch,
           'age': age_of_epoch},
    name="eeg"
)

In [ ]:
# data.to_netcdf('data/LEMON/lemon_preprocessed_for_cbramod.nc5', engine='h5netcdf')